In [1]:
from ovcg import *
from fesol import *
import numpy as np
import scipy.io as sio
from interface import DynamicUnderRelaxation, InterfaceData, RelativeCovergenceMonitor

In [2]:
k=20
# problem discription:
# basically, with a fixed Pr=0.01 and Re=500, the k is the ratio between
# kappa_s/kappa_f, where kappa stands for conductivity coefficient.
assert k == 1 or k == 5 or k == 20

# tolerance setting
tol = 1e-6
if tol > 1e-1:
    print('The tolerance is too large, this may lead to instability or low quality results')

In [3]:
# since this is a matching interface, we can create
# exact interface nodes mappings.
# this step is done offline
inodes = 100
s2f = np.zeros(inodes, dtype=int)
f2s = s2f.copy()
s2f[1] = inodes-1
start = inodes-2
for i in range(inodes-2):
    s2f[i+2] = start
    start = start - 1
count = 0
for i in range(inodes):
    f2s[s2f[i]] = i

In [4]:
# fluid solver
interface_grid = 3
interface_side = 0
interface_axis = 1

solverF = Cgins()
solverF.read_cmd('fluid_inputs/fluid_flat_plate%i_fnsd.cmd' % k)
solverF.init_grid()
solverF.init_flux_interface(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)
solverF.init_solver(debug=0, logname='fluid_outputs/dummy')
fluid_ofile = VtkFile('fluid_outputs/resFNSR%i.pvd' % k)

In [5]:
# solid solver
fixed_temp_tag = 2
adiabatic_tag = 3
interface_tag = 4

kappa = 100.0
rho = 1.0
Cp = 100.0
T_bot = 310.0

solverS = HeatSolver(kappa=kappa, rho=rho, Cp=Cp)
solverS.output_file = 'solid_outputs/resFNSR%i.pvd' % k
solverS.load_mesh('solid_inputs/solid_plate.msh')
solverS.set_initial_condition(T_bot)
solverS.define_adiabatic_bd(tag=adiabatic_tag)
solverS.define_const_temperature_bd(tag=fixed_temp_tag, value=T_bot)
solverS.define_robin_interface(tag=interface_tag)
solverS.init_solver()

In [6]:
# coupling data and setups
abTempF = InterfaceData(size=inodes, value=300.0)
abTempF2S = InterfaceData(size=inodes, value=300.0)
FluxS = InterfaceData(size=inodes, value=0.0)
FluxF = InterfaceData(size=inodes, value=0.0)

under_relax = DynamicUnderRelaxation(init_omega=1.0)
conv_mntr = RelativeCovergenceMonitor(tol=tol)

# maximum pc steps allowed
max_pc_steps = 200

# since this problem has no mesh motion and both kappa are material properties
# so the heat transfer coefficients don't vary in time
hF2S = solverF.get_heat_transfer_coeff(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)[f2s]

In [7]:
# coupling timing information, use fixed time step for now
Tfinal = 1.0
N = 100
dt = Tfinal/N
t = 0.0
avg_pc_iterations = 0
flog = open('FNSR%i.log' % k, mode='w')
flog.write('Fluid Neumann with solid Robin setting, study case: %i\n' % k)
flog.close()
flog = open('FNSR%i.log' % k, mode='a')

In [8]:
# begin to couple
for step in range(N):
    t += dt
    
    pc_counts = 0
    
    # back up solutions
    solverF.backup_current_solutions()
    solverS.backup()
    
    while True:
        pc_counts = pc_counts + 1
        
        # back up previous interface value
        FluxF.backup()
        
        # advance fluid
        solverF.advance(t=t, dt=dt)
        # retrieve fluid ambient temperature
        abTempF.curr[:] = solverF.get_ambient_temperature(
            grid=interface_grid,
            side=interface_side,
            axis=interface_axis
        )
        
        # mapping
        abTempF2S.curr[:] = abTempF.curr[f2s]
        # update solid robin interface
        solverS.update_robin_interface(
            tag=interface_tag,
            Ta=abTempF2S.curr,
            h_coeff=hF2S
        )
        
        # advance solid
        solverS.advance(t=t, dt=dt)
        # retrieve solid interface flux
        FluxS.curr[:] = solverS.get_interface_flux(tag=interface_tag)
        
        # mapping
        FluxF.curr[:] = FluxS.curr[s2f]
        
        # update residual
        FluxF.update_res()
        
        # test convergence
        is_conv = conv_mntr.determine_convergence(FluxF)
        if is_conv or pc_counts >= max_pc_steps:
            solverF.put_heat_flux_interface(
                grid=interface_grid,
                side=interface_side,
                axis=interface_axis,
                data=FluxF.curr
            )
            break
        else:
            # if not converge, then underrelaxation and update to fluid then restore
            under_relax.determine_omega(FluxF)
            under_relax.update_solution(FluxF)
            solverF.put_heat_flux_interface(
                grid=interface_grid,
                side=interface_side,
                axis=interface_axis,
                data=FluxF.curr
            )
            solverF.restore_previous_solutions()
            solverS.restore()  

    # outputs
    msg = 'step=%i, pc_iterations=%i.' % (step, pc_counts)
    print(msg)
    flog.write(msg+'\n')
    solverS.write()
    fluid_ofile.write(solverF, t)
    avg_pc_iterations = avg_pc_iterations + pc_counts
flog.close()

step=0, pc_iterations=8.
step=1, pc_iterations=6.
step=2, pc_iterations=6.
step=3, pc_iterations=5.
step=4, pc_iterations=5.
step=5, pc_iterations=5.
step=6, pc_iterations=5.
step=7, pc_iterations=4.
step=8, pc_iterations=4.
step=9, pc_iterations=4.
step=10, pc_iterations=4.
step=11, pc_iterations=4.
step=12, pc_iterations=4.
step=13, pc_iterations=4.
step=14, pc_iterations=4.
step=15, pc_iterations=4.
step=16, pc_iterations=4.
step=17, pc_iterations=4.
step=18, pc_iterations=4.
step=19, pc_iterations=4.
step=20, pc_iterations=4.
step=21, pc_iterations=4.
step=22, pc_iterations=4.
step=23, pc_iterations=4.
step=24, pc_iterations=4.
step=25, pc_iterations=4.
step=26, pc_iterations=4.
step=27, pc_iterations=4.
step=28, pc_iterations=4.
step=29, pc_iterations=4.
step=30, pc_iterations=3.
step=31, pc_iterations=3.
step=32, pc_iterations=4.
step=33, pc_iterations=3.
step=34, pc_iterations=3.
step=35, pc_iterations=3.
step=36, pc_iterations=3.
step=37, pc_iterations=3.
step=38, pc_iterations

In [9]:
theta=(solverF.get_temperature_interface(3,0,1)-300)/10
print(theta)
sio.savemat('numeric_res/kFNSR%i.mat' % k, {'theta': theta, 'avg_pc_its': avg_pc_iterations/N})

[0.85062386 0.9006286  0.91533218 0.91451726 0.91574977 0.91838852
 0.92041016 0.92222454 0.92458694 0.92581567 0.92739561 0.92858348
 0.93008961 0.93120426 0.93240174 0.93332516 0.93441396 0.93532674
 0.93611083 0.9370052  0.93772377 0.93846155 0.93914874 0.93974446
 0.94037764 0.9409059  0.94148947 0.94196314 0.94235292 0.94278646
 0.94321686 0.94356086 0.94393049 0.94425288 0.94456176 0.94482708
 0.94510671 0.9453297  0.94558983 0.94579983 0.94599147 0.94614432
 0.94630795 0.94646221 0.94660251 0.9467245  0.94684483 0.94694463
 0.94701882 0.94711128 0.94717493 0.9472352  0.94728079 0.94732042
 0.9473402  0.94737717 0.94737896 0.94737965 0.94737509 0.94735646
 0.94731164 0.9472842  0.94722912 0.94717511 0.9470982  0.94702709
 0.94692108 0.94683493 0.94670351 0.94657482 0.94646739 0.94631225
 0.94611087 0.94594422 0.94572177 0.94550563 0.94524505 0.9449799
 0.9446572  0.94436285 0.94398751 0.94361435 0.94322518 0.94280369
 0.94235389 0.94176254 0.94131809 0.94069892 0.93995595 0.93931